# SVG Stripes Background
## Goal Image:
![Goal](https://media.istockphoto.com/vectors/abstract-seamless-black-dash-lines-diagonal-pattern-on-white-vector-id1278434948)

In [ ]:
from IPython.display import display, HTML
import numpy as np
import random 
from scipy.spatial.transform import Rotation as R


In [ ]:
# SVG Writer
def writeSvg(content):
    header = '<svg width="'+str(imgsize[0])+'" height="'+str(imgsize[1])+'" xmlns="http://www.w3.org/2000/svg">\n'
    footer = '</svg>\n'
    return header+("".join([c.build(1) for c in content]))+footer

# Helpers:
def buildattr(attrs):
    ret = [k+'="'+str(attrs[k])+'"' for k in attrs.keys()]
    return ' '.join(ret)

# Classes
class SVGOBJ:
    def __init__(self,name):
        self.a = {}
        self.name = name
    def __str__(self):
        return self.build()
    def build(self, depth=0):
        # When updating, update Group.build as well.
        indent = ' '*(2*depth)
        header='<'+self.name+' '+buildattr(self.a)
        footer='/>\n'
        return indent+header+self.getContentTag()+footer
    def attr(self,*ts):
        if len(ts)==2 and isinstance(ts[0],str):
            ts = [(ts[0],ts[1])]
        for t in ts:
            self.a[str(t[0])] = str(t[1])
        return self
    def stroke(self,color):
        return self.attr('stroke',color)
    def width(self,width):
        return self.attr('stroke-width',width)
    def fill(self,color):
        return self.attr('fill',color)
class Group(SVGOBJ):
    def __init__(self, children = []):
        super().__init__("g")
        self.children = children
    def getContentTag(self):
        return ''
    def build(self, depth=0):
        # When updating, update SVGOBJ.build as well.
        indent = ' '*(2*depth)
        header='<'+self.name+' '+buildattr(self.a)+'>\n'
        footer='</'+self.name+'>\n'
        content = "".join([a.build(depth+1) for a in self.children])
        return indent+header+content+indent+footer
    def maskBy(self, mask):
        self.attr('mask','url(#'+mask.a["id"]+')')
        return self
class Rect(SVGOBJ):
    def __init__(self, x, y, w, h):
        super().__init__("rect")
        self.x = x
        self.y = y
        self.w = w
        self.h = h
    def getContentTag(self):
        return ' '+buildattr({'x':self.x,'y':self.y,'width':self.w,'height':self.h})
class Circle(SVGOBJ):
    def __init__(self, cx, cy, r):
        super().__init__("circle")
        self.cx = cx
        self.cy = cy
        self.r = r
    def getContentTag(self):
        return ' '+buildattr({'cx':self.cx,'cy':self.cy,'r':self.r})
# Path object. Common operators are:
#  - Move         ( M x y | m dx dy )
#  - Line         ( L x y | l dx dy )
#  - Horizontal   ( H x   | h dx    )
#  - Vertical     ( V y   | v dy    )
#  - Cubic Bezier ( C hx1 hy1 hx2 hy2 x y )
#  - Quadr Bezier ( Q hx1 hy1         x y )
#  - S (C cont)   ( S         hx2 hy2 x y ) #(hx1,hy1) = prev(2x-hx2,2y-hy2) aka mirror through (x,y) 
#  - T (Q cont)   ( S                 x y ) #(hx1,hy1) = prev(2x-hx1,2y-hy1) aka mirror through (x,y)
#  - Arc ( A rx ry x-axis-rotation large-arc-flag sweep-flag x y )
#  - Z (close loop to start)
class Path(SVGOBJ):
    def __init__(self, x, y):
        super().__init__("path")
        self.d = ["M",str(x),str(y)]
    def biop(self,op,*xs):
        self.d.append(op)
        for x in xs:
            self.d.append(str(x))
        return self
    def getContentTag(self):
        return ' d="'+(' '.join(self.d))+'"'
    def round(self):
        return self.attr('stroke-linecap',"round")
class Mask(Group):
    def __init__(self, id, children = []):
        super().__init__(children)
        self.name = "mask"
        self.attr('id',id)

In [ ]:
def get_size(u,l):
    t = random.random()**0.5
    return t*u+l*(1-t)

def generate_background():
    return Rect(0,0,imgsize[0],imgsize[1]).fill("#f0d981").stroke("#000").width(2)

In [ ]:
def matToTransform(size):
    t1 = np.matrix([[1,0,-size/2],[0,1,-size/2],[0,0,1]])
    rot = R.from_euler('z', angle, degrees=True).as_matrix()
    t2 = np.matrix([[1,0,imgsize[0]/2],[0,1,imgsize[1]/2],[0,0,1]])
    rmat = t2*rot*t1
    return "matrix(" +(" ".join([str(a) for a in np.array(rmat[0:2,:].transpose().flatten())[0]]))+")"


In [ ]:
def buildWhole():
    size = np.sqrt(imgsize[0]**2 + imgsize[1]**2)
    size

    # keep track of the y coordinate 
    loc_y = 0 
    ps = []
    while(loc_y < size): 
        line_width = get_size(widths[0],widths[1]) 
        loc_y += spacing/2 + line_width/2
        # keep track of the x coordinate , start at an offset
        loc_x = -size/2
        p = Path(loc_y,0)
        # randomly pick the width of the line
        p.width(str(line_width)).stroke("#f7f4e9").round().fill("None")
        # print(loc_y)
        while (loc_x < size):
            # draw a line of random size
            line_length = get_size(lengths[0], lengths[1])
            p.biop('v', line_length)
            # skip a gap of random size
            gap_length = get_size(gaps[0], gaps[1]) + float(p.a["stroke-width"])
            p.biop('m', 0, gap_length)
            # update x coordinate 
            loc_x += line_length +gap_length
        # update y coordinate
        loc_y += spacing/2 + line_width/2
        ps.append(p)

    m = Mask("boundingbox",[Rect(10,10,imgsize[0]-20,imgsize[1]-20).fill("#fff")])

    g = Group([Group(ps).attr("transform",matToTransform(size))]).maskBy(m)


    o = writeSvg([generate_background(), m, g])
    # print(o)
    display(HTML(o))

    with open('out.svg','w', encoding="utf8") as f:
        # Write the CSV header (column names seperated with a comma)
        f.write(o)


In [ ]:
imgsize = (768,768)
widths = (1.5,8)
lengths = (8,500)
gaps = (50,150)
spacing = 10
angle = 40

buildWhole()